# Chapter 9 - Dimensionality Reduction Using Feature Extraction

## 9.0 - Introduction

It is common to have access to thousands and even hundreds of thousands of features. For example, in Chapter 8 we transformed a 256 x 256-pixel color image into 196,608 features. Furthermore, because each of these pixels can take one of 256 possible values, there ends up being $ 256^{196608} $ different configurations our observation can take. This is problematic because we will practically never be able to collect enough observations to cover even a small fraction of those configurations and our learning algorithms do not have enough data to operate correctly.

Fortunately, not all features are created equal and the goal of feature extraction for dimensionality reduction is to transform our set of features, $ p_{original} $, such that we end up with a new set, $ p_{new} $, where $ p_{original} > p_{new} $, while still keeping much of the underlying information. Put another way, we reduce the number of features with only a small loss in our data's ability to generate high-quality predictions. In this chapter, we will cover a number of feature extraction techniques to do just this.

One downside of the feature extraction techniques we discuss is that the new features we generate will not be interpretable by humans. They will contain as much or nearly as much ability to train our models, but will appear to the human eye as a collection of random numbers. If we wanted to maintain our ability to interpret our models, dimensionality reduction through feature selection is a better option.

## 9.1 - Reducing Features Using Principal Components

### Problem
Given a set of features, you want to reduce the number of features while retaining the variance in the data.

### Solution
Use principle component analysis with scikit's 'PCA':

In [1]:
# Load libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import datasets

In [7]:
# Load the data
digits = datasets.load_digits()
digits.data

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [8]:
# Standardize the feature matrix
features = StandardScaler().fit_transform(digits.data)
features

array([[ 0.        , -0.33501649, -0.04308102, ..., -1.14664746,
        -0.5056698 , -0.19600752],
       [ 0.        , -0.33501649, -1.09493684, ...,  0.54856067,
        -0.5056698 , -0.19600752],
       [ 0.        , -0.33501649, -1.09493684, ...,  1.56568555,
         1.6951369 , -0.19600752],
       ...,
       [ 0.        , -0.33501649, -0.88456568, ..., -0.12952258,
        -0.5056698 , -0.19600752],
       [ 0.        , -0.33501649, -0.67419451, ...,  0.8876023 ,
        -0.5056698 , -0.19600752],
       [ 0.        , -0.33501649,  1.00877481, ...,  0.8876023 ,
        -0.26113572, -0.19600752]])

In [10]:
# Create a PCA that will retain 99% of variance
pca = PCA(n_components = 0.99, whiten = True)
pca

PCA(copy=True, iterated_power='auto', n_components=0.99, random_state=None,
    svd_solver='auto', tol=0.0, whiten=True)

In [11]:
# Conduct PCA
features_pca = pca.fit_transform(features)
features_pca

array([[ 0.70631939, -0.39512814, -1.73816236, ...,  0.36526417,
        -0.31369006,  0.05355504],
       [ 0.21732591,  0.38276482,  1.72878893, ..., -0.17818068,
        -0.14031747,  1.18179755],
       [ 0.4804351 , -0.13130437,  1.33172761, ..., -0.01924571,
        -0.23580029,  0.92966158],
       ...,
       [ 0.37732433, -0.0612296 ,  1.0879821 , ..., -1.05526847,
         1.75559618, -0.87894699],
       [ 0.39705007, -0.15768102, -1.08160094, ...,  0.10442881,
         0.65907949,  1.1292155 ],
       [-0.46407544, -0.92213976,  0.12493334, ..., -1.10593026,
         0.54434185, -0.26573597]])

In [12]:
# Show results
print("Original number of features:", features.shape[1])
print("Reduced number of features:",features_pca.shape[1])

Original number of features: 64
Reduced number of features: 54


### Discussion
Principal component analysis (PCA) is a popular linear dimensionality reduction technique. PCA projects observations onto the (hopefully fewer) principal components of the feature matrix that retain the most variance. PCA is an unsupervised technique, meaning that it does not use information from the target vector and instead only considers the feature matrix.

For a mathematical description of how PCA works, see the external resources listed at the end of this recipe. However, we can understand the intuition behind PCA using a simple example. In the following figure, our data contains two featuresm $ x_1 $ and $ x_2 $. Looking at the viualization, it should be clear that observations are spread out like a cigar, with a lot of length and very little height, More specifically, we can say that the variance of the "length" is significantly greater than the "height". Instead of length and height, we refer to the "directions" with the most variance as the first principal component and the "direction" with the second-most variance as the second principal component (and so on).

If we wanted to reduce our features, one strategy would be to project all observations on our 2D space onto the 1D principal component. We would lose the information captured in the second principal component, but in some situations that would be acceptable trade-off. This is PCA.

PCA is implementedin scikit-learn using the 'pca' method. 'n_components' has two operations, depending on the argument provided. If the argument is greater than 1, 'n_components' will return that many features. This leads to the question of how to select the number of features that is optimal. Fortunately for us, if the argument to 'n_components' is between 0 and 1, 'pca' returns the minimum amount of features that retain that much varience. It is common to use values of 0.95 and 0.99, meaning 95% and 99% of varience of the original features has been retained, respectively. 'whiten=True' transforms the values of each principal component so that they have zero mean and unit variance. Another parameter and argument is 'svd_solver="randomized"', which implements a stochastic algorithm to find the first principal components on ofetn significantly less time.

The output of our solution shows that PCA let us reduce our dimensionality by 10 features while stillretaining 99% of the information (variance) in the feature.

![features x1 vs. x2](images/featureX1vsX2.jpg)

### See Also
   * scikit-learn documentation on PCA (http://bit.ly/2FrSvyx)
   * Choosing the Number of Principal Components (http://bit.ly/2FrSGtH)
   * Principal component analysis with linear algebra (http://bit.ly/2FuzdIW)